In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
lib = %pwd
lib

'd:\\practice\\MLproject\\research'

In [4]:
import os

file_dir,file_name = os.path.split(lib)
file_dir

'd:\\practice\\MLproject'

In [5]:
import sys

sys.path.append(file_dir)

In [6]:
from src.mlProject.constants import *

In [7]:
CONFIG_FILE

WindowsPath('config/config.yaml')

In [8]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [9]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger

In [10]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = r"D:\practice\MLproject\config\config.yaml",#change according to location of the dictionary
        params_filepath = r"D:\practice\MLproject\params.yaml",#change according to location of the dictionary
        schema_filepath = r"D:\practice\MLproject\schema.yaml"):#change according to location of the dictionary

        # Convert the string to a pathlib.Path object
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)
        schema_filepath = Path(schema_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-20 19:05:03,919: INFO: common: yaml file: D:\practice\MLproject\config\config.yaml loaded successfully]
[2024-04-20 19:05:03,952: INFO: common: yaml file: D:\practice\MLproject\params.yaml loaded successfully]
[2024-04-20 19:05:03,972: INFO: common: yaml file: D:\practice\MLproject\schema.yaml loaded successfully]
[2024-04-20 19:05:03,976: INFO: common: created directory at: artifacts]
[2024-04-20 19:05:03,978: INFO: common: created directory at: artifacts/data_ingestion]


[2024-04-20 19:05:05,043: INFO: 489543768: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24857
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e72f5befdc6bbe5f0ff08c13d10e7047157387c134ad608b9200dee366dd97d8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2C24:240D4B:22EE77:2E96AE:6623C488
Accept-Ranges: bytes
Date: Sat, 20 Apr 2024 13:35:05 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10230-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1713620105.997102,VS0,VE278
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 6694f7371474446a2c356e1c8f04e718619bfe70
Expires: Sat, 20 Apr 2024 13:40:05 GMT
Source-Age: 0

]
